In [98]:
import cv2
import numpy as np

In [117]:
path1 = r'imgs\IMG_20240510_172748.jpg'
path2 = r'imgs\IMG_20240510_172837.jpg'
path3 = r'imgs\IMG_20240510_172930.jpg'

In [141]:
def getCountors(frame, draw = True):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_gray = cv2.GaussianBlur(frame_gray, ksize = (5, 5), sigmaX=1)

    frame_gray = cv2.Canny(frame_gray, threshold1=50, threshold2=125)

    cntrs, h = cv2.findContours(frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = []

    for cnt in cntrs:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*perimeter, closed = True)
        bbox = cv2.boundingRect(approx)

        cnts.append((cnt, approx, area, perimeter, bbox))

    cnts = sorted(cnts, key = lambda x: -x[2])
    
    cnt = [c[0] for c in cnts[:1]]
    # cnt = [c[0] for c in cnts]

    if draw:
        M = cv2.moments(cnt[0])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        # Draw the center of the contour
        cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)

        frame = cv2.drawContours(frame, cnt, -1, (0, 0, 255), 5)

    cnt = cnt[0]
    l = tuple(cnt[cnt[:,:,0].argmin()][0])
    r = tuple(cnt[cnt[:,:,0].argmax()][0])
    t = tuple(cnt[cnt[:,:,1].argmin()][0])
    b = tuple(cnt[cnt[:,:,1].argmax()][0])


    return frame, cnts, (l, r, t, b)

img = cv2.imread(path1)

w, h = img.shape[1]//1, img.shape[0]//1

img = cv2.resize(img, (w, h))

img_cnts, cnts1, (l, r, t, b) = getCountors(img)


cv2.imshow('Detected Circles', img_cnts)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [142]:
cv2.circle(img_cnts, l, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, r, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, t, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, b, 8, (0, 255, 0), -1)

p1 = (l[0], t[1])
p2 = (r[0], t[1])
p3 = (l[0], b[1])
p4 = (r[0], b[1])

# p_c = (int((l[0] + r[0] + t[0] + b[0])/4), int((l[1] + r[1] + t[1] + b[1])/4))

# cv2.circle(img_cnts, p_c, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p1, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p2, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p3, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p4, 8, (255, 255, 0), -1)

# Display the image
cv2.imshow('Extreme Points', img_cnts)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [143]:
pts1 = np.float32([l, t, b, r])
# pts2 = np.float32([p1, p2, p3, p4])
pts2 = np.float32([p1, p2, p3, p4])

M = cv2.getPerspectiveTransform(pts1, pts2)

result = cv2.warpPerspective(img, M, (w, h))

# cv2.circle(result, (450, 450), 8, (255, 255, 0), -1)

cv2.imshow('TR', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [145]:
cv2.imwrite('result.jpg', result)

True

In [147]:
img = cv2.imread(path2)

w, h = img.shape[1]//4, img.shape[0]//4

img = cv2.resize(img, (w, h))

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# gray_blurred = cv2.medianBlur(gray_img, 21, 1)
gray_blurred = cv2.bilateralFilter(gray_img, 15, 75, 75)
# gray_blurred = cv2.GaussianBlur(gray_img, (21, 21), 7)

cv2.imshow('Detected Circles', gray_blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20,  
                                   param1=100, param2=250, minRadius=0, maxRadius=0)

# detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20, 
#                                    param1=50, param2=220, minRadius=0, maxRadius=0)

if detected_circles is not None:
    detected_circles = np.uint16(np.around(detected_circles))
    circles = detected_circles[0, :]
    circles = sorted(circles, key=lambda x: x[2], reverse=True)
    
    # Draw only the 5 largest circles
    i = 0
    for circle in circles:
        (x, y, r) = circle
        cv2.circle(img, (x, y), r, (0, 255, 0), 2)
        cv2.circle(img, (x, y), 1, (0, 0, 255), 3)
        if i == 5: break


cv2.imshow('Detected Circles', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [149]:
import cv2
import numpy as np

def find_colored_areas(image, color_range_red, color_range_green):
  """
  Finds red and green areas in an image using OpenCV.

  Args:
      image: The input image as a NumPy array.
      color_range_red: Lower and upper bounds for red color in HSV (Hue, Saturation, Value) format.
      color_range_green: Lower and upper bounds for green color in HSV format.

  Returns:
      A tuple containing two NumPy arrays:
          - red_mask: Binary mask where white pixels represent red areas.
          - green_mask: Binary mask where white pixels represent green areas.
  """
  # Convert image to HSV colorspace
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  # Create masks for red and green colors
  red_mask = cv2.inRange(hsv, color_range_red[0], color_range_red[1])
  green_mask = cv2.inRange(hsv, color_range_green[0], color_range_green[1])

  # Optional: Refine masks (noise reduction, contour smoothing)
  # ... (implement further processing as needed)

  return red_mask, green_mask

# Example usage (adjust color ranges as needed)
image = cv2.imread(path1)  # Replace with your image path

w, h = image.shape[1]//4, image.shape[0]//4

img = cv2.resize(image, (w, h))

color_range_red = (np.array([170, 100, 100], np.uint8), np.array([180, 255, 255], np.uint8))  # Lower red limit (adjust) 
color_range_green = (np.array([40, 50, 50], np.uint8), np.array([80, 255, 255], np.uint8))  # Lower green limit (adjust)

red_mask, green_mask = find_colored_areas(image, color_range_red, color_range_green)

# Display results (optional)
cv2.imshow("Original Image", image)
cv2.imshow("Red Mask", red_mask)
cv2.imshow("Green Mask", green_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
